In [1]:
import sys
import os

import numpy as np
from process_bigraph import Composite

from biosimulator_processes import CORE


sys.path.insert(0, '..')

Available processes:
[ 'console-emitter',
  'ram-emitter',
  'composite',
  'cobra-process',
  'copasi-process',
  'tellurium-process',
  'utc-amici',
  'utc-copasi',
  'utc-tellurium',
  'smoldyn-process',
  'copasi-step',
  'tellurium-step',
  'amici-step',
  'plotter',
  'plotter2d',
  'utc-comparator',
  'simple-neuron',
  'simple-neuron-network']


In [2]:
model_fp = './examples/smoldyn/MinE.txt'
os.path.exists(model_fp)

True

In [3]:
composition = Composite(
    config={
        'state': {
            'smoldyn': {
                '_type': 'process',
                'address': 'local:smoldyn-process',
                'config': {'model_filepath': model_fp},
                'inputs': {
                    'species_counts': ['species_counts'],
                    'molecules': ['molecules']
                },
                'outputs': {
                    'species_counts': ['species_counts'],
                    'molecules': ['molecules']
                }
            }
            
        }
    }, 
    core=CORE
)

INITAL: {'MinDMinE': 0, 'MinD_ADP': 0, 'MinD_ATP': 4000, 'MinE': 1400}


In [4]:
mols = composition.state['molecules']
counts = composition.state['species_counts']

counts

{'MinDMinE': 0, 'MinD_ADP': 0, 'MinD_ATP': 4000, 'MinE': 1400}

In [5]:
composition.run(10)

ValueError: too many values to unpack (expected 3)